In [ ]:
import torch
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import pydicom
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from skimage.transform import resize
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()
FLAIR_MAX_LEN = 514
T1W_MAX_LEN = 400
T1WCE_MAX_LEN = 400
T2W_MAX_LEN = 472
VIDEO_NUMBER = 585
from matplotlib import pyplot as plt
from torch.nn import functional as torch_functional
import time


In [ ]:
!pip install efficientnet_pytorch
!conda install pytorch torchvision cudatoolkit=9.0 -c pytorch
#!pip install torchsummary
from efficientnet_pytorch import EfficientNet
#print(os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/00001'))

In [ ]:
root_dir = '../input/rsna-miccair-bain-tumor-radiogenomic-classification/'
print(len(os.listdir(root_dir + 'train')))

In [ ]:
def get_image(data, size_x = 256, size_y = 256):
    '''
    Returns the image data as a numpy array.
    '''  
    if np.max(data.pixel_array)==0:
        img = data.pixel_array
    else:
        img = data.pixel_array/np.max(data.pixel_array)
        img = (img * 255).astype(np.uint8)
    return resize(img, (size_x, size_y))

data = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-24.dcm')
img = get_image(data)
#print(img[128])
img = resize(img, (256, 256))
print(np.max(img))
plt.imshow(img, cmap='gray')

In [ ]:
max_len = 0
for i in tqdm(os.listdir((root_dir + 'train/'))):
        l = len(os.listdir(root_dir + 'train/' + i + '/' + 'T2w'))
        if l > max_len:
            max_len = l
print(max_len)

In [ ]:
root_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
def load_data(root_dir, batch_size, number, answers_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv',
              size_x = 256, size_y = 256):
    # returns 4 numpy arrays of padded video(4 dim) and array of answers
    video_dir = os.listdir(root_dir)
    video_dir.sort()
    video_number = len(video_dir)
    arrays_size = batch_size
    if number * batch_size > video_number:
        raise Exception('out of range')
    if number * batch_size + batch_size > video_number:
        arrays_size = video_number - number * batch_size
    x = np.full((arrays_size, FLAIR_MAX_LEN, size_x, size_y), 0.0)
    y = np.full((arrays_size, T1W_MAX_LEN, size_x, size_y), 0.0)
    z = np.full((arrays_size, T1WCE_MAX_LEN, size_x, size_y), 0.0)
    w = np.full((arrays_size, T2W_MAX_LEN, size_x, size_y), 0.0)
    answers = np.full((arrays_size), 0.0)
    df = pd.read_csv(answers_dir)
    for i in range(number * batch_size, number * batch_size + arrays_size):
        dir_flair = os.listdir(root_dir + video_dir[i] + '/FLAIR')
        dir_flair.sort()
        dir_t1w = os.listdir(root_dir + video_dir[i] + '/T1w')
        dir_t1w.sort()
        dir_t1wc = os.listdir(root_dir + video_dir[i] + '/T1wCE')
        dir_t1wc.sort()
        dir_t2w = os.listdir(root_dir + video_dir[i] + '/T2w')
        dir_t2w.sort()
        answers[i - number * batch_size] = int(df[df['BraTS21ID'] == int(video_dir[i])]['MGMT_value'])
        #print(root_dir + video_dir[i])
        for j in range(len(dir_flair)):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/FLAIR/' + dir_flair[j])
            x[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(len(dir_t1w)):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T1w/' + dir_t1w[j])
            y[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(len(dir_t1wc)):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T1wCE/' + dir_t1wc[j])
            z[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(len(dir_t2w)):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T2w/' + dir_t2w[j])
            w[i - number * batch_size][j] = get_image(data, size_x, size_y)
    return x, y, z, w, answers

def load_data_resized(root_dir, batch_size, number, answers_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv',
              size_x = 256, size_y = 256, resize_FLAIR_MAX_LEN = 514, resize_T1W_MAX_LEN = 400, resize_T1WCE_MAX_LEN = 400, resize_T2W_MAX_LEN = 472):
    # returns 4 numpy arrays of padded video(4 dim) and array of answers
    # resize output data
    video_dir = os.listdir(root_dir)
    video_dir.sort()
    #print(video_dir)
    video_number = len(video_dir)
    arrays_size = batch_size
    if number * batch_size > video_number:
        raise Exception('out of range')
    if number * batch_size + batch_size > video_number:
        arrays_size = video_number - number * batch_size
    x = np.full((arrays_size, resize_FLAIR_MAX_LEN, size_x, size_y), 0.0)
    y = np.full((arrays_size, resize_T1W_MAX_LEN, size_x, size_y), 0.0)
    z = np.full((arrays_size, resize_T1WCE_MAX_LEN, size_x, size_y), 0.0)
    w = np.full((arrays_size, resize_T2W_MAX_LEN, size_x, size_y), 0.0)
    answers = np.full((arrays_size), 0.0)
    df = pd.read_csv(answers_dir)
    for i in range(number * batch_size, number * batch_size + arrays_size):
        #print((root_dir + video_dir[i] +'/FLAIR'))
        dir_flair = os.listdir(root_dir + video_dir[i] +'/FLAIR')
        dir_flair.sort()
        dir_t1w = os.listdir(root_dir + video_dir[i] + '/T1w')
        dir_t1w.sort()
        dir_t1wc = os.listdir(root_dir + video_dir[i] + '/T1wCE')
        dir_t1wc.sort()
        dir_t2w = os.listdir(root_dir + video_dir[i] + '/T2w')
        dir_t2w.sort()
        answers[i - number * batch_size] = int(df[df['BraTS21ID'] == int(video_dir[i])]['MGMT_value'])
        #print(int(df[df['BraTS21ID'] == int(video_dir[i])]['MGMT_value']))
        #print(root_dir + video_dir[i])
        for j in range(min(resize_FLAIR_MAX_LEN, len(dir_flair))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/FLAIR/' + dir_flair[j])
            x[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(min(resize_T1W_MAX_LEN, len(dir_t1w))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T1w/' + dir_t1w[j])
            y[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(min(resize_T1WCE_MAX_LEN, len(dir_t1wc))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T1wCE/' + dir_t1wc[j])
            z[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(min(resize_T2W_MAX_LEN, len(dir_t2w))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T2w/' + dir_t2w[j])
            w[i - number * batch_size][j] = get_image(data, size_x, size_y)
    return x, y, z, w, answers


def transform_array(numpy_array, new_len = 32):
    l = []
    for i in range(new_len, len(numpy_array[0])):
        l.append(i)
    return np.delete(numpy_array, l, 1)


In [ ]:
x, y, z, w, answers = load_data_resized(root_dir, 1, 0, resize_FLAIR_MAX_LEN = 32,
                                                resize_T1W_MAX_LEN = 32, resize_T1WCE_MAX_LEN = 32, resize_T2W_MAX_LEN = 32)

#plt.imshow(x[0][0], cmap='gray')
plt.imshow(x[0][2], cmap='gray')

In [ ]:
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification' + '/train_labels.csv')
df[df['BraTS21ID'] == 2]

In [ ]:
'''class Video_preprocess(nn.Module):
    def __init__(self):
        super(Video_preprocess, self).__init__()
        self.Flatten = nn.Flatten()
        self.conv_process = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=16),
            nn.Tanh(),
            nn.MaxPool2d(4, 4),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=32),
            nn.Sigmoid(),
            nn.MaxPool2d(4, 4)
        )
        self.gru1 = nn.GRU(input_size=49, hidden_size=256, num_layers=3, batch_first=True, dropout=0.1, bidirectional=True)
        #self.fcx = nn.Linear(262144, 64200)
    def forward(self, x):
        #print(x.size())
        x = self.conv_process(x)
        x = torch.flatten(x, start_dim=2, end_dim=-1)
        #print(x.size(0))
        h0 = torch.zeros(3 * 2, x.size(0), 256).to(device)

        x, h0 = self.gru1(x, h0)
        #print(x)
        return x, h0



#test_video_preprocess = Video_preprocess()
#test_tensor = torch.rand(1, 128, 256, 256)
#test_video_preprocess(test_tensot)
#test_video_preprocess(test_tensor)







class Video_classifier(nn.Module):
    def __init__(self, FLAIR_MAX_LEN = 514, T1W_MAX_LEN = 400, T1WCE_MAX_LEN = 400, T2W_MAX_LEN = 472, size_x = 256, size_y = 256):
        super(Video_classifier, self).__init__()
        self.FLAIR_MAX_LEN = FLAIR_MAX_LEN
        self.T1W_MAX_LEN = T1W_MAX_LEN
        self.T1WCE_MAX_LEN = T1WCE_MAX_LEN
        self.T2W_MAX_LEN = T2W_MAX_LEN
        self.size_x = size_x
        self.size_y = size_y


        self.Flair_extraction = Video_preprocess()
        self.T1w_extraction = Video_preprocess()
        self.T1wce_extraction = Video_preprocess()
        self.T2w_extraction = Video_preprocess()

        self.GRU = nn.GRU(input_size=2048, hidden_size=1024, num_layers=3, bidirectional=True, dropout=0.1, batch_first=True)
        self.Flatten = nn.Flatten()
        self.fc1 = nn.Linear(1048576, 128)
        #self.fc2 = nn.Linear(128000, 16000)
        #self.fc3 = nn.Linear(2048, 64)
        #self.fc4 = nn.Linear(256, 64)
        self.fc5 = nn.Linear(128, 1)


    def forward(self, x, y, z, w):
        x, hx = self.Flair_extraction(x)
        y, hy = self.T1w_extraction(y)
        z, hz = self.T1wce_extraction(z)
        w, hw = self.T2w_extraction(w)
        #print(torch.numel(x) * 4)
        #print(x.size())
        h0 = torch.cat((hx, hy, hz, hw), 2).to(device)
        x = torch.cat((x, y, z, w), 2).to(device)
        #print(x.size())
        #print(h0.size())
        x, h0 = self.GRU(x, h0)
        x = self.Flatten(x)
        #print(x.size())
        x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        #x = torch.sigmoid(self.fc3(x))
        #x = F.tanh(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))
        return x
'''


class soft_max_linear(nn.Module):
    def __init__(self):
        super(soft_max_linear, self).__init__()
        self.fc = nn.Linear(2560, 2)

    def forward(self, x):
        x = self.fc(x)
        x = F.softmax(x, dim = 1)
        return x


class TripleFrameProccess(nn.Module):
    def __init__(self):
        super(TripleFrameProccess, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b7')
        self.model._fc = soft_max_linear()

    def forward(self, x):
        if torch.max(x) == 0:
            l = []
            for i in range(len(x)):
                l.append([-1 , -1])
            return torch.tensor(l).to(device)
        x = self.model(x)
        return x

class FolderPrediction(nn.Module):
    def __init__(self):
        super(FolderPrediction, self).__init__()
        self.triple_frame_proccess = TripleFrameProccess()
        self.BLSTM = nn.LSTM(input_size=4, hidden_size=64, num_layers=3, bidirectional=True, batch_first=True, dropout=0.1)

    def forward(self, x):
        x = x.unsqueeze(2).to(device)
        # for now work only whit batch == 1
        answers = torch.tensor([[]]).to(device);
        for i in range(0,len(x[0]), 3):
            y = torch.cat((torch.clone(x[0][i]), torch.clone(x[0][i + 1]),torch.clone(x[0][i + 2])), dim=0).to(device)
            y = y.unsqueeze(0).to(device)
            #print(y.size())
            y = self.triple_frame_proccess(y)
            answers = torch.cat((answers, y), dim=1).to(device)
        #print(answers.size())
        answers = torch.reshape(answers, (1, 2, 4)).to(device)
        h0 = torch.zeros(3 * 2, answers.size(0), 64).to(device)
        c0 = torch.zeros(3 * 2, answers.size(0), 64).to(device)
        answers, (h0, c0) = self.BLSTM(answers, (h0, c0))
        return answers, (h0, c0);



class TumorDetector(nn.Module):
    def __init__(self):
        super(TumorDetector, self).__init__()
        self.Flair_extraction = FolderPrediction()
        self.T1w_extraction = FolderPrediction()
        self.T1wce_extraction = FolderPrediction()
        self.T2w_extraction = FolderPrediction()
        self.BLSTM = nn.LSTM(input_size=128, hidden_size=256, num_layers=3, bidirectional=True, dropout=0.1, batch_first=True)
        self.fc1 = nn.Linear(4096, 256)
        self.fc2 = nn.Linear(256, 1)


    def forward(self, x, y, z, w):
        x, (x0, cx0) = self.Flair_extraction(x)
        y, (y0, cy0) = self.T1w_extraction(y)
        z, (z0, cz0) = self.T1w_extraction(z)
        w, (w0, cw0) = self.T2w_extraction(w)
        #print(x.size())
        x = torch.cat((x, y, z, w), dim=1).to(device)
        h0 = torch.cat((x0, y0, z0, w0), dim=2).to(device)
        c0 = torch.cat((cx0, cy0, cw0, cz0), dim=2).to(device)
        #print(h0.size())
        #print(x.size())
        x, (h0, c0) = self.BLSTM(x, (h0, c0))
        x = torch.flatten(x, start_dim=1).to(device)
        #print(x.size())
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


In [ ]:
model = TumorDetector()
#model._fc = soft_max_linear(model._fc)
model.to(device)
criterion = torch_functional.binary_cross_entropy_with_logits
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
NUM_EPOCKS = 1
BATCH_SIZE = 1
VAL_NUMBER = 32


def convert_value(x):
    #print(x.shape)
    if x < 0.5:
        return 0.0
    return 1.0

def print_acuracy(outputs, answers):
    acuracy = 0
    for i in range(len(answers)):
        if (answers[i]) == convert_value((outputs[i])):
            acuracy += 1
    print("Acuracy == ", acuracy / VAL_NUMBER)
    print(outputs)

def acuracy_chek(model):
    acuracy = 0
    for batch_number in tqdm(range(VAL_NUMBER // BATCH_SIZE)):
        x, y, z, w, answers = load_data_resized(root_dir, BATCH_SIZE, batch_number,size_x = 224, size_y = 224
                                                , resize_FLAIR_MAX_LEN = 12, resize_T1W_MAX_LEN = 12, resize_T1WCE_MAX_LEN = 12, resize_T2W_MAX_LEN = 12)
        #x = np.expand_dims(x, axis=1)
        #y = np.expand_dims(y, axis=1)
        #z = np.expand_dims(z, axis=1)
        #w = np.expand_dims(w, axis=1)
        x = torch.from_numpy(x).to(device=device, dtype=torch.float)
        y = torch.from_numpy(y).to(device=device, dtype=torch.float)
        z = torch.from_numpy(z).to(device=device, dtype=torch.float)
        w = torch.from_numpy(w).to(device=device, dtype=torch.float)
        #x = torch.cat((x, y, z, w), 2).to(device)
        answers = np.expand_dims(answers, axis=1)
        answers = torch.from_numpy(answers).to(device=device, dtype=torch.float)
        #answers = F.one_hot(answers.to(torch.int64), num_classes=2)
        #answers = answers.to(torch.float)
        outputs = model(x, y, z, w)
        answers = answers.to("cpu").detach().numpy()
        outputs = outputs.to("cpu").detach().numpy()
        plt.plot(answers, label = "answers")
        plt.plot(outputs, label = "prediction")
        plt.show()
        print(outputs)
        for i in range(len(answers)):
            if (answers[i]) == convert_value((outputs[i])):
                acuracy += 1
        
    print("Acuracy == ", acuracy / VAL_NUMBER)
    

#model.eval()
#acuracy_chek(model)
    
    
for epoch in range(NUM_EPOCKS):
    running_loss = 0.0
    model.train()
    for batch_number in tqdm(range(VAL_NUMBER // BATCH_SIZE, VIDEO_NUMBER // BATCH_SIZE)):
        
        
        x, y, z, w, answers = load_data_resized(root_dir, BATCH_SIZE, batch_number,size_x = 224, size_y = 224
                                                , resize_FLAIR_MAX_LEN = 12, resize_T1W_MAX_LEN = 12, resize_T1WCE_MAX_LEN = 12, resize_T2W_MAX_LEN = 12)
        answers = np.expand_dims(answers, axis=1)
        #x = np.expand_dims(x, axis=1)
        #y = np.expand_dims(y, axis=1)
        #z = np.expand_dims(z, axis=1)
        #w = np.expand_dims(w, axis=1)
        #print(np.max(x))
        x = torch.from_numpy(x).to(device=device, dtype=torch.float)
        y = torch.from_numpy(y).to(device=device, dtype=torch.float)
        z = torch.from_numpy(z).to(device=device, dtype=torch.float)
        w = torch.from_numpy(w).to(device=device, dtype=torch.float)
        answers = torch.from_numpy(answers).to(device=device, dtype=torch.float)
        
        #answers = F.one_hot(answers.to(torch.int64), num_classes=2)
        #answers = answers.to(torch.float)
        #x = torch.cat((x, y, z, w), 2).to(device)
        outputs = model(x, y, z, w)
        #print(outputs.size())
        #print(torch.max(x))
        loss = criterion(outputs, answers)
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
        if batch_number % 5 == 0:
            print("loss == ", running_loss / 5)
            running_loss = 0.0
        # print statistics
    model.eval()
    acuracy_chek(model)
torch.save(model, "eff_net_model.ph")

In [ ]:
def load_data_for_prediction(root_dir, batch_size, number, size_x = 256, size_y = 256,
                             resize_FLAIR_MAX_LEN = 514, resize_T1W_MAX_LEN = 400, resize_T1WCE_MAX_LEN = 400, resize_T2W_MAX_LEN = 472):
    video_dir = os.listdir(root_dir)
    video_dir.sort()
    # print(video_dir)
    video_number = len(video_dir)
    arrays_size = batch_size
    if number * batch_size > video_number:
        raise Exception('out of range')
    if number * batch_size + batch_size > video_number:
        arrays_size = video_number - number * batch_size
    x = np.full((arrays_size, resize_FLAIR_MAX_LEN, size_x, size_y), 0.0)
    y = np.full((arrays_size, resize_T1W_MAX_LEN, size_x, size_y), 0.0)
    z = np.full((arrays_size, resize_T1WCE_MAX_LEN, size_x, size_y), 0.0)
    w = np.full((arrays_size, resize_T2W_MAX_LEN, size_x, size_y), 0.0)
    #answers = np.full((arrays_size), 0.0)
    for i in range(number * batch_size, number * batch_size + arrays_size):
        # print((root_dir + video_dir[i] +'/FLAIR'))
        dir_flair = os.listdir(root_dir + video_dir[i] + '/FLAIR')
        dir_flair.sort()
        dir_t1w = os.listdir(root_dir + video_dir[i] + '/T1w')
        dir_t1w.sort()
        dir_t1wc = os.listdir(root_dir + video_dir[i] + '/T1wCE')
        dir_t1wc.sort()
        dir_t2w = os.listdir(root_dir + video_dir[i] + '/T2w')
        dir_t2w.sort()
        # print(int(df[df['BraTS21ID'] == int(video_dir[i])]['MGMT_value']))
        # print(root_dir + video_dir[i])
        for j in range(min(resize_FLAIR_MAX_LEN, len(dir_flair))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/FLAIR/' + dir_flair[j])
            x[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(min(resize_T1W_MAX_LEN, len(dir_t1w))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T1w/' + dir_t1w[j])
            y[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(min(resize_T1WCE_MAX_LEN, len(dir_t1wc))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T1wCE/' + dir_t1wc[j])
            z[i - number * batch_size][j] = get_image(data, size_x, size_y)
        for j in range(min(resize_T2W_MAX_LEN, len(dir_t2w))):
            data = pydicom.dcmread(root_dir + video_dir[i] + '/T2w/' + dir_t2w[j])
            w[i - number * batch_size][j] = get_image(data, size_x, size_y)
    return x, y, z, w, video_dir



def make_prediction(model, batch_size, tests_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/'):
    tests = os.listdir(tests_dir)
    tests.sort()
    predictions = {'BraTS21ID' : [],
                   'MGMT_value' : []
                   }
    for i in tqdm(range(len(tests) // batch_size)):
        x, y, z, w, video_dir = load_data_for_prediction(tests_dir, batch_size, i)
        x = torch.from_numpy(x).to(device=device, dtype=torch.float)
        y = torch.from_numpy(y).to(device=device, dtype=torch.float)
        z = torch.from_numpy(z).to(device=device, dtype=torch.float)
        w = torch.from_numpy(w).to(device=device, dtype=torch.float)
        x = torch.cat((x, y, z, w), 2).to(device)
        outputs = model(x)
        outputs = outputs.tolist()
        outputs = list(map(lambda x : x[1], outputs))
        predictions['BraTS21ID'] += video_dir
        predictions['MGMT_value'] += outputs
    df = pd.DataFrame(predictions)
    print(df.head)
    df.to_csv("sumbmission.csv", index=False)

    
make_prediction(model, 1)